# ITU 5G Energy Consumption Challenge - GitData

<font color="red">Please make a copy of this Jupyter Notebook to your own Google Drive or local machine before changing anything.</font>

This Jupyter Notebook will also be updated regularly on GitHub: https://github.com/GitData-GA/itu5g/tree/main/code/ITU_5G_Energy_Consumption_Challenge.ipynb

In [1]:
# Import all libraries here
import random
random.seed(88)
import pandas as pd
import numpy as np
np.random.seed(88)
import tensorflow as tf
tf.random.set_seed(88)
from tensorflow import keras
keras.utils.set_random_seed(88)
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import mean_absolute_percentage_error
from matplotlib import pyplot as plt
from keras import backend as K

In [2]:
# Output format
# Description: The file contains days, hours, and base stations for which we
# need energy consumption estimation and will be used for scoring predictions.
power_pred = pd.read_csv('https://data.gd.edu.kg/bwz7t527/power_consumption_prediction.csv')
# Dataset 1
# Description: Energy consumption data (ECdata.csv): include hour-level energy
# consumption specifications (e.g., total energy consumption of the base stations).
ECdata = pd.read_csv('https://data.gd.edu.kg/bwz7t527/ECdata.csv')
# Dataset 2
# Description: Cell-level data (CLdata.csv): include hour-level counters,
# including service compliance counters (e.g., load) and energy-saving methods
# counters (e.g., duration of energy saving mode activation).
CLdata = pd.read_csv('https://data.gd.edu.kg/bwz7t527/CLdata.csv')
# Dataset 3
# Description: Base Station basic information (BSinfo.csv): include
# configuration parameters and hardware attributes
BSinfo = pd.read_csv('https://data.gd.edu.kg/bwz7t527/BSinfo.csv')

In [3]:
data = ECdata.merge(CLdata, on=["Time", "BS"], how="inner")
data = data.merge(BSinfo, on="BS", how="inner")
data = data.sort_values(by='Time', ascending=True)
# Convert date strings to datetime objects
data['Time'] = pd.to_datetime(data['Time'])

# Sort the dataset by date
data.sort_values(by='Time', inplace=True)

# Extract target variable 'Energy'
y = data['Energy']

# Drop unnecessary columns (e.g., 'Time', 'Energy')
data.drop(columns=['Time', 'Energy'], inplace=True)

# One-hot encode categorical features
categorical_cols = ['BS', 'CellName_x', 'CellName_y', 'RUType', 'Mode']
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# Standardize numerical features
scaler = StandardScaler()
data_encoded[data_encoded.columns] = scaler.fit_transform(data_encoded[data_encoded.columns])

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data_encoded, y, test_size=0.2, shuffle=False)

# Convert data to numpy arrays
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [4]:
X_train_lstm = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_lstm = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
lstm_model = keras.Sequential([
    layers.LSTM(units=80, recurrent_dropout=0.25),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu', kernel_regularizer=l1(0.001)),
    layers.Dense(units=1)
])

# Compile the model
lstm_model.compile(optimizer='adam', loss='mean_absolute_percentage_error')

# Train the model
history = lstm_model.fit(X_train_lstm, y_train, epochs=30, batch_size=256, validation_split=0.2)

# Fit model on validation set and get error
y_pred = lstm_model.predict(X_test_lstm)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {mape:.5f}%')

Epoch 1/30
315/315 [==============================] - 11s 23ms/step - loss: 39.8543 - val_loss: 11.2527
Epoch 2/30
315/315 [==============================] - 8s 25ms/step - loss: 13.3034 - val_loss: 8.4462
Epoch 3/30
315/315 [==============================] - 8s 24ms/step - loss: 11.7689 - val_loss: 7.8651
Epoch 4/30
315/315 [==============================] - 7s 23ms/step - loss: 10.9960 - val_loss: 7.3471
Epoch 5/30
315/315 [==============================] - 8s 26ms/step - loss: 10.4534 - val_loss: 7.0466
Epoch 6/30
315/315 [==============================] - 7s 21ms/step - loss: 9.9957 - val_loss: 6.8803
Epoch 7/30
315/315 [==============================] - 8s 26ms/step - loss: 9.5944 - val_loss: 6.8209
Epoch 8/30
315/315 [==============================] - 7s 22ms/step - loss: 9.3725 - val_loss: 6.7166
Epoch 9/30
315/315 [==============================] - 9s 29ms/step - loss: 9.1812 - val_loss: 6.7139
Epoch 10/30
315/315 [==============================] - 8s 25ms/step - loss: 8.9959 -

In [5]:
model = keras.Sequential([
    layers.Dense(units=64, activation='relu', kernel_regularizer=l1(0.001)),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu', kernel_regularizer=l1(0.001)),
    layers.Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_percentage_error')

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=256, validation_split=0.2)

# Fit model on validation set and get error
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {mape:.5f}%')

Epoch 1/30
315/315 [==============================] - 4s 10ms/step - loss: 38.9391 - val_loss: 12.9080
Epoch 2/30
315/315 [==============================] - 2s 7ms/step - loss: 16.3237 - val_loss: 11.3475
Epoch 3/30
315/315 [==============================] - 2s 7ms/step - loss: 15.1062 - val_loss: 10.7160
Epoch 4/30
315/315 [==============================] - 2s 7ms/step - loss: 14.2980 - val_loss: 10.2147
Epoch 5/30
315/315 [==============================] - 3s 9ms/step - loss: 13.4861 - val_loss: 9.6683
Epoch 6/30
315/315 [==============================] - 3s 9ms/step - loss: 12.7800 - val_loss: 9.4610
Epoch 7/30
315/315 [==============================] - 2s 7ms/step - loss: 12.1817 - val_loss: 9.1928
Epoch 8/30
315/315 [==============================] - 2s 7ms/step - loss: 11.8398 - val_loss: 9.3522
Epoch 9/30
315/315 [==============================] - 2s 7ms/step - loss: 11.6785 - val_loss: 9.1384
Epoch 10/30
315/315 [==============================] - 2s 8ms/step - loss: 11.5997 - v